In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('air_quality').getOrCreate()

In [2]:
df = spark.read.csv('2015_Air_quality_in_northern_Taiwan.csv', inferSchema=True, header=True)
df.printSchema()

root
 |-- time: string (nullable = true)
 |-- station: string (nullable = true)
 |-- AMB_TEMP: double (nullable = true)
 |-- CH4: double (nullable = true)
 |-- CO: double (nullable = true)
 |-- NO: double (nullable = true)
 |-- NO2: double (nullable = true)
 |-- NOx: double (nullable = true)
 |-- O3: double (nullable = true)
 |-- PM10: integer (nullable = true)
 |-- PM2.5: double (nullable = true)
 |-- RH: double (nullable = true)
 |-- SO2: double (nullable = true)
 |-- THC: double (nullable = true)
 |-- UVB: double (nullable = true)
 |-- WS_HR: double (nullable = true)
 |-- WD_HR: double (nullable = true)



In [3]:
df.show()

+--------------+-------+--------+---+----+---+----+----+----+----+-----+----+----+---+---+-----+-----+
|          time|station|AMB_TEMP|CH4|  CO| NO| NO2| NOx|  O3|PM10|PM2.5|  RH| SO2|THC|UVB|WS_HR|WD_HR|
+--------------+-------+--------+---+----+---+----+----+----+----+-----+----+----+---+---+-----+-----+
| 2015-1-1 0:00|Banqiao|    16.0|2.1|0.79|1.2|16.0|17.0|37.0| 177| null|57.0|12.0|2.2|0.0|  4.2| 69.0|
| 2015-1-1 1:00|Banqiao|    16.0|2.1| 0.8|1.3|16.0|17.0|36.0| 178| null|57.0|11.0|2.2|0.0|  4.0| 67.0|
| 2015-1-1 2:00|Banqiao|    16.0|2.1|0.71|1.0|13.0|14.0|38.0| 163| null|57.0| 8.0|2.2|0.0|  3.5| 63.0|
| 2015-1-1 3:00|Banqiao|    15.0|2.0|0.66|0.8|11.0|12.0|39.0| 147| null|58.0| 6.5|2.2|0.0|  3.3| 63.0|
| 2015-1-1 4:00|Banqiao|    15.0|2.0|0.53|0.6|10.0|11.0|38.0| 131| null|58.0| 5.5|2.1|0.0|  3.1| 69.0|
| 2015-1-1 5:00|Banqiao|    14.0|2.0| 0.5|0.8|11.0|12.0|37.0| 112| null|57.0| 5.7|2.1|0.0|  3.3| 68.0|
| 2015-1-1 6:00|Banqiao|    14.0|2.0|0.57|1.4|18.0|19.0|29.0| 103| null|5

In [4]:
df_renamed = df.withColumnRenamed("PM2.5","Target")
df_renamed.columns

['time',
 'station',
 'AMB_TEMP',
 'CH4',
 'CO',
 'NO',
 'NO2',
 'NOx',
 'O3',
 'PM10',
 'Target',
 'RH',
 'SO2',
 'THC',
 'UVB',
 'WS_HR',
 'WD_HR']

In [5]:
df_renamed.describe([
'time',
 'station',
 'AMB_TEMP',
 'CH4',
 'CO',
 'NO'
]).show()

+-------+-------------+-------+------------------+------------------+------------------+-----------------+
|summary|         time|station|          AMB_TEMP|               CH4|                CO|               NO|
+-------+-------------+-------+------------------+------------------+------------------+-----------------+
|  count|       218640| 218640|            199609|             94425|            214895|           213711|
|   mean|         null|   null|23.323307365900323|1.9338141382037757|0.5631571232462226|9.024569629078387|
| stddev|         null|   null| 5.861835725704358|0.2118078992555375|0.5398304692750239|17.71515020450054|
|    min|2015-1-1 0:00|Banqiao|             -30.0|              -0.8|             -0.06|             -8.5|
|    max|2015-9-9 9:00|   izhi|              56.0|               9.9|              38.0|            358.0|
+-------+-------------+-------+------------------+------------------+------------------+-----------------+



In [6]:
df_renamed.describe([
 'NO2',
 'NOx',
 'O3',
 'PM10',
 'Target'
]).show()

+-------+------------------+------------------+------------------+------------------+------------------+
|summary|               NO2|               NOx|                O3|              PM10|            Target|
+-------+------------------+------------------+------------------+------------------+------------------+
|  count|            213711|            213712|            197940|            212971|            212783|
|   mean|17.877133605663666|26.884046286591317|29.033217641709406| 43.74847749224073| 19.14470516911596|
| stddev|12.071730697352447|26.653123489558798|17.854042863093966|103.99935752069378|24.490165998624523|
|    min|              -3.2|              -2.2|              -8.5|               -17|             -18.0|
|    max|             139.0|             411.0|             144.0|              5004|             985.0|
+-------+------------------+------------------+------------------+------------------+------------------+



In [7]:
df_renamed.describe([
 'RH',
 'SO2',
 'THC',
 'UVB',
 'WS_HR',
 'WD_HR'
]).show()

+-------+------------------+------------------+-------------------+------------------+------------------+------------------+
|summary|                RH|               SO2|                THC|               UVB|             WS_HR|             WD_HR|
+-------+------------------+------------------+-------------------+------------------+------------------+------------------+
|  count|            199493|            212970|              94426|             25868|            181789|            182712|
|   mean| 75.92471565418337|3.4175682020942038| 2.2016372609238593|1.3230052574609346|1.9917959832552752|145.21905184114894|
| stddev|13.479893549707922|   5.7190402805654|0.38449657460101944| 2.329660032822547|1.7260317377483954| 86.71776551201917|
|    min|               0.0|              -7.8|               -0.3|               0.0|               0.0|               0.0|
|    max|             100.0|             332.0|               10.0|              13.0|              50.0|             360.0|


In [8]:
df_renamed.show()
df_renamed.count()

+--------------+-------+--------+---+----+---+----+----+----+----+------+----+----+---+---+-----+-----+
|          time|station|AMB_TEMP|CH4|  CO| NO| NO2| NOx|  O3|PM10|Target|  RH| SO2|THC|UVB|WS_HR|WD_HR|
+--------------+-------+--------+---+----+---+----+----+----+----+------+----+----+---+---+-----+-----+
| 2015-1-1 0:00|Banqiao|    16.0|2.1|0.79|1.2|16.0|17.0|37.0| 177|  null|57.0|12.0|2.2|0.0|  4.2| 69.0|
| 2015-1-1 1:00|Banqiao|    16.0|2.1| 0.8|1.3|16.0|17.0|36.0| 178|  null|57.0|11.0|2.2|0.0|  4.0| 67.0|
| 2015-1-1 2:00|Banqiao|    16.0|2.1|0.71|1.0|13.0|14.0|38.0| 163|  null|57.0| 8.0|2.2|0.0|  3.5| 63.0|
| 2015-1-1 3:00|Banqiao|    15.0|2.0|0.66|0.8|11.0|12.0|39.0| 147|  null|58.0| 6.5|2.2|0.0|  3.3| 63.0|
| 2015-1-1 4:00|Banqiao|    15.0|2.0|0.53|0.6|10.0|11.0|38.0| 131|  null|58.0| 5.5|2.1|0.0|  3.1| 69.0|
| 2015-1-1 5:00|Banqiao|    14.0|2.0| 0.5|0.8|11.0|12.0|37.0| 112|  null|57.0| 5.7|2.1|0.0|  3.3| 68.0|
| 2015-1-1 6:00|Banqiao|    14.0|2.0|0.57|1.4|18.0|19.0|29.0| 10

218640

In [17]:
df_new=df_renamed.na.drop(thresh=15)
df_new.show()
df_new.count()

+--------------+-------+--------+---+----+---+----+----+----+----+------+----+----+---+---+-----+-----+
|          time|station|AMB_TEMP|CH4|  CO| NO| NO2| NOx|  O3|PM10|Target|  RH| SO2|THC|UVB|WS_HR|WD_HR|
+--------------+-------+--------+---+----+---+----+----+----+----+------+----+----+---+---+-----+-----+
| 2015-1-1 0:00|Banqiao|    16.0|2.1|0.79|1.2|16.0|17.0|37.0| 177|  null|57.0|12.0|2.2|0.0|  4.2| 69.0|
| 2015-1-1 1:00|Banqiao|    16.0|2.1| 0.8|1.3|16.0|17.0|36.0| 178|  null|57.0|11.0|2.2|0.0|  4.0| 67.0|
| 2015-1-1 2:00|Banqiao|    16.0|2.1|0.71|1.0|13.0|14.0|38.0| 163|  null|57.0| 8.0|2.2|0.0|  3.5| 63.0|
| 2015-1-1 3:00|Banqiao|    15.0|2.0|0.66|0.8|11.0|12.0|39.0| 147|  null|58.0| 6.5|2.2|0.0|  3.3| 63.0|
| 2015-1-1 4:00|Banqiao|    15.0|2.0|0.53|0.6|10.0|11.0|38.0| 131|  null|58.0| 5.5|2.1|0.0|  3.1| 69.0|
| 2015-1-1 5:00|Banqiao|    14.0|2.0| 0.5|0.8|11.0|12.0|37.0| 112|  null|57.0| 5.7|2.1|0.0|  3.3| 68.0|
| 2015-1-1 6:00|Banqiao|    14.0|2.0|0.57|1.4|18.0|19.0|29.0| 10

75897

In [19]:
df_new.describe([
 'CH4',
 'THC',
 'UVB',
]).show()

+-------+-------------------+------------------+------------------+
|summary|                CH4|               THC|               UVB|
+-------+-------------------+------------------+------------------+
|  count|              75779|             75779|             16998|
|   mean| 1.9098470552519518|2.1266155531216837|1.2774326391340138|
| stddev|0.19153453390056688|0.3243939789825188|2.2942451513042834|
|    min|               -0.7|              -0.3|               0.0|
|    max|                6.8|               7.8|              13.0|
+-------+-------------------+------------------+------------------+



In [20]:
df_filter=df_new.drop('time','station','UVB')
df_filter.show()

from pyspark.sql.functions  import mean
mean_amb = df_filter.select(mean(df_filter['AMB_TEMP'])).collect()
mean_amb
mean_amb_val = mean_amb[0][0]
a = df_filter.na.fill(mean_amb_val, subset=['AMB_TEMP'])

mean_ch4 = a.select(mean(a['CH4'])).collect()
mean_ch4
mean_ch4_val = mean_ch4[0][0]
b = a.na.fill(mean_ch4_val, subset=['CH4'])

mean_co = b.select(mean(b['CO'])).collect()
mean_co
mean_co_val = mean_co[0][0]
c = b.na.fill(mean_co_val, subset=['CO'])

mean_no = c.select(mean(c['NO'])).collect()
mean_no
mean_no_val = mean_no[0][0]
d = c.na.fill(mean_no_val, subset=['NO'])

mean_no2 = d.select(mean(d['NO2'])).collect()
mean_no2
mean_no2_val = mean_no2[0][0]
e = d.na.fill(mean_no2_val, subset=['NO2'])

mean_nox = e.select(mean(e['NOx'])).collect()
mean_nox
mean_nox_val = mean_nox[0][0]
f = e.na.fill(mean_nox_val, subset=['NOx'])

mean_o3 = f.select(mean(f['O3'])).collect()
mean_o3
mean_o3_val = mean_o3[0][0]
g = f.na.fill(mean_o3_val, subset=['O3'])

mean_pm10 = g.select(mean(g['PM10'])).collect()
mean_pm10
mean_pm10_val = mean_pm10[0][0]
h = g.na.fill(mean_pm10_val, subset=['PM10'])

mean_target = h.select(mean(h['Target'])).collect()
mean_target
mean_target_val = mean_target[0][0]
i = h.na.fill(mean_target_val, subset=['Target'])

mean_rh = i.select(mean(i['RH'])).collect()
mean_rh
mean_rh_val = mean_rh[0][0]
j = i.na.fill(mean_rh_val, subset=['RH'])

mean_so2 = j.select(mean(j['SO2'])).collect()
mean_so2
mean_so2_val = mean_so2[0][0]
k = j.na.fill(mean_so2_val, subset=['SO2'])

mean_thc = k.select(mean(k['THC'])).collect()
mean_thc
mean_thc_val = mean_thc[0][0]
l = k.na.fill(mean_thc_val, subset=['THC'])

mean_ws = l.select(mean(l['WS_HR'])).collect()
mean_ws
mean_ws_val = mean_ws[0][0]
m = l.na.fill(mean_ws_val, subset=['WS_HR'])

mean_wd = m.select(mean(m['WD_HR'])).collect()
mean_wd
mean_wd_val = mean_wd[0][0]
n = m.na.fill(mean_wd_val, subset=['WD_HR'])

n.show()


+--------+---+----+---+----+----+----+----+------+----+----+---+-----+-----+
|AMB_TEMP|CH4|  CO| NO| NO2| NOx|  O3|PM10|Target|  RH| SO2|THC|WS_HR|WD_HR|
+--------+---+----+---+----+----+----+----+------+----+----+---+-----+-----+
|    16.0|2.1|0.79|1.2|16.0|17.0|37.0| 177|  null|57.0|12.0|2.2|  4.2| 69.0|
|    16.0|2.1| 0.8|1.3|16.0|17.0|36.0| 178|  null|57.0|11.0|2.2|  4.0| 67.0|
|    16.0|2.1|0.71|1.0|13.0|14.0|38.0| 163|  null|57.0| 8.0|2.2|  3.5| 63.0|
|    15.0|2.0|0.66|0.8|11.0|12.0|39.0| 147|  null|58.0| 6.5|2.2|  3.3| 63.0|
|    15.0|2.0|0.53|0.6|10.0|11.0|38.0| 131|  null|58.0| 5.5|2.1|  3.1| 69.0|
|    14.0|2.0| 0.5|0.8|11.0|12.0|37.0| 112|  null|57.0| 5.7|2.1|  3.3| 68.0|
|    14.0|2.0|0.57|1.4|18.0|19.0|29.0| 103|  null|57.0| 5.8|2.1|  2.2| 74.0|
|    14.0|2.0|0.61|2.3|17.0|20.0|27.0| 104|  null|56.0| 6.5|2.2|  3.0| 66.0|
|    14.0|2.0|0.59|2.9|15.0|18.0|29.0| 111|  null|53.0| 6.8|2.1|  3.3| 71.0|
|    14.0|2.0| 0.6|4.1|16.0|20.0|29.0| 111|  null|50.0| 5.9|2.1|  3.2| 73.0|

In [21]:
n.count()


75897

In [22]:
df_final=n.na.drop( )
df_final.count()

75897

In [19]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [20]:
assembler = VectorAssembler(
    inputCols=[ 'AMB_TEMP','CH4','CO','NO','NO2','NOx','O3','PM10','RH','SO2','THC','WS_HR','WD_HR'],
    outputCol="features")

In [21]:
output = assembler.transform(df_final)
output.printSchema()
output.head(1)

root
 |-- AMB_TEMP: double (nullable = false)
 |-- CH4: double (nullable = false)
 |-- CO: double (nullable = false)
 |-- NO: double (nullable = false)
 |-- NO2: double (nullable = false)
 |-- NOx: double (nullable = false)
 |-- O3: double (nullable = false)
 |-- PM10: integer (nullable = true)
 |-- Target: double (nullable = false)
 |-- RH: double (nullable = false)
 |-- SO2: double (nullable = false)
 |-- THC: double (nullable = false)
 |-- WS_HR: double (nullable = false)
 |-- WD_HR: double (nullable = false)
 |-- features: vector (nullable = true)



[Row(AMB_TEMP=16.0, CH4=2.1, CO=0.79, NO=1.2, NO2=16.0, NOx=17.0, O3=37.0, PM10=177, Target=19.410086405453264, RH=57.0, SO2=12.0, THC=2.2, WS_HR=4.2, WD_HR=69.0, features=DenseVector([16.0, 2.1, 0.79, 1.2, 16.0, 17.0, 37.0, 177.0, 57.0, 12.0, 2.2, 4.2, 69.0]))]

In [22]:
final_data = output.select("features",'Target')
final_data.show()

+--------------------+------------------+
|            features|            Target|
+--------------------+------------------+
|[16.0,2.1,0.79,1....|19.410086405453264|
|[16.0,2.1,0.8,1.3...|19.410086405453264|
|[16.0,2.1,0.71,1....|19.410086405453264|
|[15.0,2.0,0.66,0....|19.410086405453264|
|[15.0,2.0,0.53,0....|19.410086405453264|
|[14.0,2.0,0.5,0.8...|19.410086405453264|
|[14.0,2.0,0.57,1....|19.410086405453264|
|[14.0,2.0,0.61,2....|19.410086405453264|
|[14.0,2.0,0.59,2....|19.410086405453264|
|[14.0,2.0,0.6,4.1...|19.410086405453264|
|[15.0,2.0,0.55,4....|19.410086405453264|
|[14.0,2.0,0.51,3....|19.410086405453264|
|[15.0,2.0,0.47,4....|19.410086405453264|
|[14.0,2.0,0.42,2....|19.410086405453264|
|[14.0,2.0,0.41,3....|19.410086405453264|
|[13.0,2.0,0.42,2....|19.410086405453264|
|[13.0,2.0,0.45,1....|19.410086405453264|
|[13.0,2.0,0.46,2....|19.410086405453264|
|[13.0,2.0,0.45,2....|19.410086405453264|
|[13.0,2.0,0.48,2....|19.410086405453264|
+--------------------+------------

In [23]:
train_data,test_data = final_data.randomSplit([0.7,0.3])
train_data.describe().show()
test_data.describe().show()

+-------+------------------+
|summary|            Target|
+-------+------------------+
|  count|             53005|
|   mean|19.408395586103882|
| stddev|17.598041913023984|
|    min|             -18.0|
|    max|             941.0|
+-------+------------------+

+-------+------------------+
|summary|            Target|
+-------+------------------+
|  count|             22892|
|   mean| 19.41400139233163|
| stddev|14.939272549199789|
|    min|             -15.0|
|    max|             912.0|
+-------+------------------+



In [58]:
lr = LinearRegression(labelCol='Target')
# Fit the model to the data.
lrModel = lr.fit(train_data)
# Print the coefficients and intercept for linear regression.
print("Intercept: {}".format(lrModel.intercept))
print("Coefficients: {}".format(lrModel.coefficients))


Intercept: -3.2811332203152315
Coefficients: [-0.22877184735304304,2.7847486941358426,8.856760411081673,-0.21579305128717452,0.14979557842592583,-0.05531361725468201,0.13733534638417444,0.1918673052684453,-0.015392070611222624,0.8268687405534041,1.5070808028475056,-0.24654492603831105,0.010656857486337112]


In [39]:
test_results = lrModel.evaluate(test_data)
print ('Model: Linear Regression')
# evaluation metrics 
print("R2: {}".format(test_results.r2))
print("MAE: {}".format(test_results.meanAbsoluteError))
print("MSE: {}".format(test_results.meanSquaredError))
print("RSME: {}".format(test_results.rootMeanSquaredError))
# This shows the difference between the predicted value and the test data.
test_results.residuals.show()

Model: Linear Regression
R2: 0.4696396418853753
MAE: 6.601228560114828
MSE: 118.3616428116236
RSME: 10.879413716355472
+--------------------+
|           residuals|
+--------------------+
|  15.795063938246614|
| -0.6015010719740843|
|  26.370124632660655|
|  13.108645845766098|
|  -4.493409862068312|
|  -3.662990593586265|
|   4.060409204997754|
|  23.389310578166807|
|    32.5415171442926|
|  12.496772412361508|
| -11.210649252602567|
| -12.705244608242252|
|  -2.669170042781971|
|   6.623894336002081|
|  -12.40047152868543|
|-0.01630768274092...|
|  -5.118850963303764|
|  -5.453941137172988|
|  -5.710952328850874|
|  -4.382052112879322|
+--------------------+
only showing top 20 rows



In [59]:
from pyspark.ml.classification import (RandomForestClassifier,LogisticRegression,DecisionTreeClassifier)
from pyspark.ml import Pipeline

from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Target", outputCol="Target_index").fit(final_data)
df_end = indexer.transform(final_data)
df_end.show()

+--------------------+------------------+------------+
|            features|            Target|Target_index|
+--------------------+------------------+------------+
|[16.0,2.1,0.79,1....|19.410086405453264|        33.0|
|[16.0,2.1,0.8,1.3...|19.410086405453264|        33.0|
|[16.0,2.1,0.71,1....|19.410086405453264|        33.0|
|[15.0,2.0,0.66,0....|19.410086405453264|        33.0|
|[15.0,2.0,0.53,0....|19.410086405453264|        33.0|
|[14.0,2.0,0.5,0.8...|19.410086405453264|        33.0|
|[14.0,2.0,0.57,1....|19.410086405453264|        33.0|
|[14.0,2.0,0.61,2....|19.410086405453264|        33.0|
|[14.0,2.0,0.59,2....|19.410086405453264|        33.0|
|[14.0,2.0,0.6,4.1...|19.410086405453264|        33.0|
|[15.0,2.0,0.55,4....|19.410086405453264|        33.0|
|[14.0,2.0,0.51,3....|19.410086405453264|        33.0|
|[15.0,2.0,0.47,4....|19.410086405453264|        33.0|
|[14.0,2.0,0.42,2....|19.410086405453264|        33.0|
|[14.0,2.0,0.41,3....|19.410086405453264|        33.0|
|[13.0,2.0

In [60]:
(trainingData, testData) = df_end.randomSplit([0.7, 0.3])

In [65]:
rf = RandomForestClassifier(labelCol="Target_index", featuresCol="features", numTrees=20)
model_rf = rf.fit(trainingData)

In [66]:
prediction_rf = model_rf.transform(testData)
prediction_rf.show()

+--------------------+------+------------+--------------------+--------------------+----------+
|            features|Target|Target_index|       rawPrediction|         probability|prediction|
+--------------------+------+------------+--------------------+--------------------+----------+
|[8.6,2.0,0.5,0.9,...|  37.0|        38.0|[0.62137461235917...|[0.03106873061795...|       9.0|
|[8.8,1.9,0.34,2.0...|   7.0|        13.0|[1.43322467495765...|[0.07166123374788...|       0.0|
|[9.4,1.9,0.57,5.6...|  13.0|         3.0|[1.39764832583661...|[0.06988241629183...|       0.0|
|[9.4,2.0,0.44,1.5...|  47.0|        49.0|[0.25718130279135...|[0.01285906513956...|      17.0|
|[9.5,2.0,0.32,1.1...|  31.0|        31.0|[0.80862595708760...|[0.04043129785438...|       1.0|
|[9.5,2.1,0.67,28....|  46.0|        43.0|[0.33530554781614...|[0.01676527739080...|      26.0|
|[9.6,2.0,0.48,0.5...|  36.0|        36.0|[0.70405818559863...|[0.03520290927993...|       9.0|
|[9.6,2.1,0.38,9.6...|  19.0|         9.

In [67]:
prediction_rf.select("prediction", "features").show()

+----------+--------------------+
|prediction|            features|
+----------+--------------------+
|       9.0|[8.6,2.0,0.5,0.9,...|
|       0.0|[8.8,1.9,0.34,2.0...|
|       0.0|[9.4,1.9,0.57,5.6...|
|      17.0|[9.4,2.0,0.44,1.5...|
|       1.0|[9.5,2.0,0.32,1.1...|
|      26.0|[9.5,2.1,0.67,28....|
|       9.0|[9.6,2.0,0.48,0.5...|
|       9.0|[9.6,2.1,0.38,9.6...|
|       9.0|[9.6,2.1,0.4,1.3,...|
|       1.0|[9.7,2.0,0.55,1.5...|
|      36.0|[9.7,2.1,0.59,12....|
|       1.0|[9.8,1.9,0.3,1.9,...|
|      17.0|[9.8,1.9,0.5,3.2,...|
|      17.0|[9.8,2.0,0.36,1.2...|
|       9.0|[9.8,2.0,0.43,1.6...|
|       2.0|[9.9,1.8,0.2,1.2,...|
|      36.0|[9.9,2.0,0.67,1.9...|
|       2.0|[10.0,1.8,0.15,0....|
|       1.0|[10.0,1.9,0.16,0....|
|       4.0|[10.0,1.9,0.17,1....|
+----------+--------------------+
only showing top 20 rows



In [68]:
lr = LogisticRegression(labelCol="Target_index", featuresCol="features")
model_lr = lr.fit(trainingData)

In [69]:
prediction_lr = model_rf.transform(testData)
prediction_lr.show()

+--------------------+------+------------+--------------------+--------------------+----------+
|            features|Target|Target_index|       rawPrediction|         probability|prediction|
+--------------------+------+------------+--------------------+--------------------+----------+
|[8.6,2.0,0.5,0.9,...|  37.0|        38.0|[0.62137461235917...|[0.03106873061795...|       9.0|
|[8.8,1.9,0.34,2.0...|   7.0|        13.0|[1.43322467495765...|[0.07166123374788...|       0.0|
|[9.4,1.9,0.57,5.6...|  13.0|         3.0|[1.39764832583661...|[0.06988241629183...|       0.0|
|[9.4,2.0,0.44,1.5...|  47.0|        49.0|[0.25718130279135...|[0.01285906513956...|      17.0|
|[9.5,2.0,0.32,1.1...|  31.0|        31.0|[0.80862595708760...|[0.04043129785438...|       1.0|
|[9.5,2.1,0.67,28....|  46.0|        43.0|[0.33530554781614...|[0.01676527739080...|      26.0|
|[9.6,2.0,0.48,0.5...|  36.0|        36.0|[0.70405818559863...|[0.03520290927993...|       9.0|
|[9.6,2.1,0.38,9.6...|  19.0|         9.

In [70]:
prediction_lr.select("prediction", "features").show()

+----------+--------------------+
|prediction|            features|
+----------+--------------------+
|       9.0|[8.6,2.0,0.5,0.9,...|
|       0.0|[8.8,1.9,0.34,2.0...|
|       0.0|[9.4,1.9,0.57,5.6...|
|      17.0|[9.4,2.0,0.44,1.5...|
|       1.0|[9.5,2.0,0.32,1.1...|
|      26.0|[9.5,2.1,0.67,28....|
|       9.0|[9.6,2.0,0.48,0.5...|
|       9.0|[9.6,2.1,0.38,9.6...|
|       9.0|[9.6,2.1,0.4,1.3,...|
|       1.0|[9.7,2.0,0.55,1.5...|
|      36.0|[9.7,2.1,0.59,12....|
|       1.0|[9.8,1.9,0.3,1.9,...|
|      17.0|[9.8,1.9,0.5,3.2,...|
|      17.0|[9.8,2.0,0.36,1.2...|
|       9.0|[9.8,2.0,0.43,1.6...|
|       2.0|[9.9,1.8,0.2,1.2,...|
|      36.0|[9.9,2.0,0.67,1.9...|
|       2.0|[10.0,1.8,0.15,0....|
|       1.0|[10.0,1.9,0.16,0....|
|       4.0|[10.0,1.9,0.17,1....|
+----------+--------------------+
only showing top 20 rows



In [71]:
dt = DecisionTreeClassifier(labelCol="Target_index", featuresCol="features")
model_dt = dt.fit(trainingData)

In [72]:
prediction_dt = model_dt.transform(testData)
prediction_dt.show()

+--------------------+------+------------+--------------------+--------------------+----------+
|            features|Target|Target_index|       rawPrediction|         probability|prediction|
+--------------------+------+------------+--------------------+--------------------+----------+
|[8.6,2.0,0.5,0.9,...|  37.0|        38.0|[24.0,45.0,31.0,4...|[0.01030485186775...|      21.0|
|[8.8,1.9,0.34,2.0...|   7.0|        13.0|[136.0,110.0,112....|[0.06879109762266...|       0.0|
|[9.4,1.9,0.57,5.6...|  13.0|         3.0|[214.0,90.0,228.0...|[0.08828382838283...|       2.0|
|[9.4,2.0,0.44,1.5...|  47.0|        49.0|[15.0,38.0,24.0,2...|[0.00567107750472...|      30.0|
|[9.5,2.0,0.32,1.1...|  31.0|        31.0|[24.0,45.0,31.0,4...|[0.01030485186775...|      21.0|
|[9.5,2.1,0.67,28....|  46.0|        43.0|[13.0,18.0,10.0,1...|[0.01229895931882...|      17.0|
|[9.6,2.0,0.48,0.5...|  36.0|        36.0|[24.0,45.0,31.0,4...|[0.01030485186775...|      21.0|
|[9.6,2.1,0.38,9.6...|  19.0|         9.

In [73]:
prediction_dt.select("prediction", "features").show()

+----------+--------------------+
|prediction|            features|
+----------+--------------------+
|      21.0|[8.6,2.0,0.5,0.9,...|
|       0.0|[8.8,1.9,0.34,2.0...|
|       2.0|[9.4,1.9,0.57,5.6...|
|      30.0|[9.4,2.0,0.44,1.5...|
|      21.0|[9.5,2.0,0.32,1.1...|
|      17.0|[9.5,2.1,0.67,28....|
|      21.0|[9.6,2.0,0.48,0.5...|
|       9.0|[9.6,2.1,0.38,9.6...|
|      21.0|[9.6,2.1,0.4,1.3,...|
|       1.0|[9.7,2.0,0.55,1.5...|
|      30.0|[9.7,2.1,0.59,12....|
|      21.0|[9.8,1.9,0.3,1.9,...|
|      25.0|[9.8,1.9,0.5,3.2,...|
|      21.0|[9.8,2.0,0.36,1.2...|
|      21.0|[9.8,2.0,0.43,1.6...|
|       4.0|[9.9,1.8,0.2,1.2,...|
|      65.0|[9.9,2.0,0.67,1.9...|
|       8.0|[10.0,1.8,0.15,0....|
|       1.0|[10.0,1.9,0.16,0....|
|       4.0|[10.0,1.9,0.17,1....|
+----------+--------------------+
only showing top 20 rows



In [74]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator 

In [75]:
rf_accuracy = MulticlassClassificationEvaluator(labelCol='Target_index', metricName='accuracy').evaluate(prediction_rf)
print("RF's accuracy is %f"%rf_accuracy)
lr_accuracy = MulticlassClassificationEvaluator(labelCol='Target_index', metricName='accuracy').evaluate(prediction_lr)
print("LR's accuracy is %f"%lr_accuracy)
dt_accuracy= MulticlassClassificationEvaluator(labelCol='Target_index', metricName='accuracy').evaluate(prediction_dt)
print("DT's accuracy is %f"%dt_accuracy)

RF's accuracy is 0.068351
LR's accuracy is 0.068351
DT's accuracy is 0.067647


In [76]:
rf_precision = MulticlassClassificationEvaluator(labelCol='Target_index', metricName='weightedPrecision').evaluate(prediction_rf)
print("RF's precision is %f"%rf_precision)
lr_precision = MulticlassClassificationEvaluator(labelCol='Target_index', metricName='weightedPrecision').evaluate(prediction_lr)
print("LR's precision is %f"%lr_precision)
dt_precision= MulticlassClassificationEvaluator(labelCol='Target_index', metricName='weightedPrecision').evaluate(prediction_dt)
print("DT's precision is %f"%dt_precision)

RF's precision is 0.058758
LR's precision is 0.058758
DT's precision is 0.039691


In [77]:
rf_auc = BinaryClassificationEvaluator(labelCol='Target_index').evaluate(prediction_rf)
print("RF's precision is %f"%rf_auc)
lr_auc = BinaryClassificationEvaluator(labelCol='Target_index').evaluate(prediction_lr)
print("LR's precision is %f"%lr_auc)
dt_auc= BinaryClassificationEvaluator(labelCol='Target_index').evaluate(prediction_dt)
print("DT's precision is %f"%dt_auc)

RF's precision is 0.390047
LR's precision is 0.390047
DT's precision is 0.417030


In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns;sns.set()
from sklearn.preprocessing import scale

ImportError: No module named 'sklearn'

In [ ]:
data3 = pd.DataFrame(scale(data2.drop(['Target'],axis=1)))
data3.columns = data2.drop(['Target'],axis=1).columns
add = data2[['Target']]
add = add.reset_index()
add = add.iloc[:,1:]
newdata = pd.concat([data3,add],axis=1)
newdata = newdata[newdata['Target']>=0]
newdata['Target'] = np.log2(newdata['PM2.5']+1)

#Variable correlation analysis
corr = newdata.corr()
fig = plt.figure(figsize=(9,6))
sns.heatmap(corr)
plt.show()
corr['Target']